In [24]:
# Importing dependencies
import os
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import pprint

In [25]:
# Path to source JSON
businessJson=os.path.join('sourceData', 'business.json')

In [26]:
# Path to Yelp food/restaurant categories csv
yelpCategories=os.path.join('sourceData', 'yelpCategories.csv')

In [27]:
# Creating pd dataframe
business_raw=pd.read_json(businessJson, lines=True)

In [28]:
# Select only the businesses in Ontario
business_on=business_raw.loc[business_raw['state'] == 'ON']

In [29]:
# Dropping any rows with blank values in these categories
business=business_on.dropna(subset=['name', 'address', 'postal_code', 'city', 'state', 'latitude', 'longitude', 'attributes',
                                               'categories', 'hours']).reset_index(drop=True)

In [30]:
# Regex to fix spelling mistakes 
business.replace({'city': {'^AGINCOURT$': 'Agincourt',
                           '^Bradford West Gwillimbury$': 'Bradford',
                           '^East Ajax$': 'Ajax',
                           '^Caledon.{,8}$': 'Caledon',
                           '^East Gwil{1,2}imbury$': 'East Gwillimbury',
                           '(?i)^.*icoke$': 'Etobicoke',
                           '^.{,9}Toro?nto.{,9}$': 'Toronto',
                           'Malton': 'Mississauga',
                           '^.{,5}Missis{1,2}a?ua?g.{1,2}$': 'Mississauga',
                           '^Regional Municipality of York$': 'North York',
                           '(?i)^North.{0,2}York$': 'North York',
                           '^York Regional Municipality$': 'York',
                           '^Willowdale$': 'North York',
                           '^North of Brampton$': 'Brampton',
                           '(?i)^Oak.?ridges$': 'Oak Ridges',
                           '^oakville$': 'Oakville',
                           '(?i)^Richmond?.?Hill?$': 'Richmond Hill',
                           '^.{,8}Scar.?bo?rough$': 'Scarborough',
                           '^.{,11}Stouffville$': 'Stouffville',
                           '(?i)^Thornhil{,2}$': 'Thornhill',
                           '^.*Vaugh.{,3}$': 'Vaughan',
                           '^Wh.?i.?by$': 'Whitby'}}, inplace=True, regex=True)

In [31]:
# Only taking these columns
business=business.loc[:, ['name', 'address', 'postal_code', 'city', 'state', 'latitude', 'longitude','categories', 'stars', 'hours', 'attributes']]
business.columns=['Name', 'Address', 'Postal_code', 'City', 'Province', 'Latitude', 'Longitude', 'Categories', 'Stars', 'Hours', 'Attributes']

In [32]:
# Turning the hours column in to a df
hours_raw=json_normalize(data=business['Hours'])
business.drop(columns='Hours', inplace=True)
# hours_raw.head()

## Handling the hours column

In [33]:
# Reorganise columns
hours_raw=hours_raw.loc[:,['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']]

In [34]:
# Create a new df with opening and closing hours
columns=hours_raw.columns
hours=hours_raw

In [35]:
# Loop through and split the columns
for column in columns:
    hours[[f"{column}_open", f"{column}_close"]]=hours_raw[column].str.split('-', expand=True)
hours.drop(columns=columns, inplace=True)
hours=hours.apply(lambda x: x.str.strip())

In [36]:
# Add hours column to the original DF
business=business.join(hours)

## Handling the categories column

In [37]:
# Turning the Categories column in to a df and adding restaurant id
category = business['Categories'].str.split(',', expand=True)
business.drop(columns='Categories', inplace=True)
category['Restaurant_id']=business.index
category=pd.melt(category, id_vars='Restaurant_id', value_name='Category').drop(columns='variable').sort_values('Restaurant_id')

In [38]:
# Eliminating None categories
category=category.loc[~category['Category'].isin([None])].reset_index(drop=True)
category['Category'] = category['Category'].str.strip()

In [39]:
# Read csv with list of yelp categories
foodCategories=pd.read_csv(yelpCategories)
foodCategories['Category'] = foodCategories['Category'].str.strip()

In [40]:
# Take only the categories from the list of yelp categories
categoryFiltered=pd.DataFrame(category.loc[category['Category'].isin(foodCategories['Category'])])

In [41]:
# Filter out categories that don't apply
categoryFiltered=categoryFiltered.loc[category['Category'].isin(['Acai Bowls', 'Afghan', 'African', 'Arabian', 'Argentine', 'Asian Fusion', 'Australian', 'Austrian', 'Bagels', 'Bangladeshi',
                                                                   'Barbeque', 'Basque', 'Beer Bar', 'Beer Gardens', 'Beer Hall', 'Belgian', 'Bistros', 'Brasseries', 'Brazilian', 'Breakfast & Brunch',
                                                                   'Brewpubs', 'British', 'Bubble Tea', 'Buffets', 'Burgers', 'Burmese', 'Cafes', 'Cajun/Creole', 'Cambodian', 'Cantonese', 'Caribbean',
                                                                   'Champagne Bars', 'Cheese Shops', 'Cheesesteaks', 'Chicken Shop', 'Chicken Wings', 'Chinese', 'Chocolatiers & Shops',
                                                                   'Cocktail Bars', 'Coffee & Tea', 'Colombian', 'Comfort Food', 'Creperies', 'Cuban', 'Cupcakes',
                                                                   'Czech', 'Czech/Slovakian', 'Delicatessen', 'Delis', 'Desserts', 'Dim Sum', 'Diners', 'Dive Bars', 'Do-It-Yourself Food', 'Donairs',
                                                                   'Egyptian', 'Ethical Grocery', 'Ethiopian', 'Falafel', 'Fast Food', 'Filipino', 'Fish & Chips', 'Fondue',
                                                                   'Food Stands', 'Food Trucks', 'French', 'Fruits & Veggies', 'Gastropubs', 'Gelato', 'German', 'Gluten-Free', 'Greek', 'Hainan', 'Haitian',
                                                                   'Hakka', 'Halal', 'Hawaiian', 'Himalayan/Nepalese', 'Hong Kong Style Cafe', 'Hot Dogs', 'Hot Pot', 'Hungarian', 'Iberian',
                                                                   'Ice Cream & Frozen Yogurt', 'Imported Food', 'Indian', 'Indonesian', 'International', 'Internet Cafes', 'Irish', 'Irish Pub', 'Italian',
                                                                   'Izakaya', 'Japanese', 'Japanese Sweets', 'Juice Bars & Smoothies', 'Kebab', 'Kombucha', 'Korean', 'Kosher', 'Laotian', 'Latin American',
                                                                   'Lebanese', 'Live/Raw Food', 'Lounges', 'Macarons', 'Malaysian', 'Mauritius', 'Mediterranean', 'Mexican', 'Middle Eastern',
                                                                   'Milkshake Bars', 'Minho', 'Modern European', 'Mongolian', 'Moroccan', 'Nicaraguan', 'Noodles', 'Pakistani', 'Pan Asian', 'Pasta Shops',
                                                                   'Persian/Iranian', 'Peruvian', 'Pizza', 'Poke', 'Polish', 'Portuguese',
                                                                   'Poutineries', 'Pubs', 'Ramen', 'Reunion', 'Russian', 'Salad', 'Salvadoran', 'Sandwiches', 'Scandinavian', 'Scottish', 'Seafood',
                                                                   'Seafood Markets', 'Shanghainese', 'Shaved Ice', 'Shaved Snow', 'Singaporean', 'Slovakian', 'Smokehouse', 'Soul Food', 'Soup',
                                                                   'South African', 'Southern', 'Spanish', 'Specialty Food', 'Sports Bars', 'Sri Lankan', 'Steakhouses', 'Street Vendors', 'Supper Clubs',
                                                                   'Sushi Bars', 'Swiss Food', 'Syrian', 'Szechuan', 'Tacos', 'Taiwanese', 'Tapas Bars', 'Tapas/Small Plates', 'Tea Rooms', 'Tempura',
                                                                   'Teppanyaki', 'Tex-Mex', 'Thai', 'Themed Cafes', 'Tiki Bars', 'Tonkatsu', 'Trinidadian', 'Turkish', 'Udon', 'Ukrainian', 'Vegan',
                                                                   'Vegetarian', 'Venezuelan', 'Vietnamese', 'Waffles', 'Whiskey Bars', 'Wine Bars', 'Wraps'])]

In [42]:
# Creating a list of unique food categories
uniqueCategories=categoryFiltered['Category'].unique()
uniqueCategories.sort()

In [43]:
# Replace all the values in categoryFiltered with the uniqueCategories index
for index, value in enumerate(uniqueCategories):
    categoryFiltered['Category'].replace(value, str(index), inplace=True)

In [44]:
# Collect all the categories to one string
categoryPivot=categoryFiltered.pivot('Restaurant_id', 'Category', 'Category')
categoryPivot['Categories']=categoryPivot.apply(lambda x: ','.join(x.dropna().values), axis=1)

In [45]:
# Join categories column on business
business=business.join(categoryPivot['Categories'], how='inner').reset_index(drop=True)
business.rename(columns={'Categories':'Category_ids'}, inplace=True)

## Handling the attributes df

In [66]:
# Turning the Attributes column in to a df and adding restaurant id
attributesRaw=json_normalize(business['Attributes'])
# business.drop(columns='Attributes', inplace=True)
attributesRaw['Restaurant_id']=business.index
attributesRaw.head(1)

,AcceptsInsurance,AgesAllowed,Alcohol,Ambience,BYOBCorkage,BestNights,BikeParking,BusinessAcceptsBitcoin,BusinessAcceptsCreditCards,BusinessParking,...,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,Restaurant_id
0,NaN,NaN,u'full_bar',"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,False,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",...,False,True,2,True,True,True,NaN,NaN,u'no',0


In [67]:
# Pull out the ambience column
ambience = attributesRaw['Ambience'].str.split(',', expand=True)
# attributesRaw.drop(columns='Ambience', inplace=True)

,0,1,2,3,4,5,6,7,8
0,{'romantic': False,'intimate': False,'classy': False,'hipster': False,'divey': False,'touristy': False,'trendy': False,'upscale': False,'casual': True}
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,{'romantic': False,'intimate': False,'classy': False,'hipster': False,'divey': False,'touristy': False,'trendy': False,'upscale': False,'casual': True}
3,{'touristy': False,'hipster': False,'romantic': False,'intimate': False,'trendy': False,'upscale': False,'classy': False,'casual': False},None
4,{'romantic': False,'intimate': False,'classy': False,'hipster': False,'divey': False,'touristy': False,'trendy': False,'upscale': False,'casual': True}
5,{'romantic': False,'intimate': False,'classy': False,'hipster': False,'divey': False,'touristy': False,'trendy': False,'upscale': False,'casual': False}
6,{'romantic': False,'intimate': False,'classy': False,'hipster': False,'divey': False,'touristy': False,'trendy': False,'upscale': False,'casual': True}
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,{'romantic': False,'intimate': False,'classy': False,'hipster': False,'divey': False,'touristy': False,'trendy': False,'upscale': False,'casual': False}
9,{'touristy': False,'hipster': False,'romantic': False,'intimate': False,'trendy': False,'upscale': False,'classy': False,'casual': False},None


In [ ]:
# Keeping only the relevant columns
attributes=attributesRaw.loc[:, ['Restaurant_id', 'Alcohol', 'Ambience', 'BestNights', 'BikeParking', 'BusinessAcceptsCreditCards', 'BusinessParking', 'ByAppointmentOnly', 'DietaryRestrictions',
                                 'DogsAllowed', 'GoodForKids', 'GoodForMeal', 'HappyHour', 'HasTV', 'Music', 'NoiseLevel', 'OutdoorSeating', 'RestaurantsAttire', 'RestaurantsGoodForGroups',
                                 'RestaurantsPriceRange2', 'RestaurantsReservations', 'RestaurantsTableService', 'RestaurantsTakeOut', 'Smoking', 'WheelchairAccessible', 'WiFi']]
attributes.head()

In [ ]:
attributes['RestaurantsCounterService'].unique()

In [ ]:
# Filter ByAppointmentOnly=false

In [ ]:
# Keeping only the relevant columns
attributes=attributesRaw.loc[:, ['Alcohol', 'Ambience', 'BestNights', 'BikeParking', 'BusinessAcceptsCreditCards', 'BusinessParking', 'ByAppointmentOnly', 'DietaryRestrictions', 'DogsAllowed',
                                 'GoodForKids', 'GoodForMeal', 'HappyHour', 'HasTV', 'Music', 'NoiseLevel', 'OutdoorSeating', 'RestaurantsAttire', 'RestaurantsGoodForGroups', 'RestaurantsPriceRange2',
                                 'RestaurantsReservations', 'RestaurantsTableService', 'RestaurantsTakeOut', 'Smoking', 'WheelchairAccessible', 'WiFi']]

## Filtering business